In [9]:
!pip install nltk

In [12]:
!pip install spacy

In [239]:
import nltk
import spacy
import pandas as pd
import numpy as np
import tensorflow as tf
import random

from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [95]:
MAX_NUMBER_OF_WORDS_PER_SENTENCE = 45
MAX_NUMBER_OF_SENTENCES = 10
MAX_NUMBER_OF_LETTERS = 550
VOCAB_SIZE = 800

In [3]:
dataset_csv = "data.csv"

In [186]:
data_frame = pd.read_csv(dataset_csv)
data_frame

,Problem,Difficulty,Tags,Worst Complexity,Link
0,Given an array of integers nums and an integer...,Easy,Brute force;Hashmap,N^2,https://leetcode.com/problems/two-sum/
1,You are given two non-empty linked lists repre...,Medium,Linked List;Implementation,N,https://leetcode.com/problems/add-two-numbers/
2,"Given a string s, find the length of the longe...",Medium,Sliding Window;Hashmap,N,https://leetcode.com/problems/longest-substrin...
3,Given two sorted arrays nums1 and nums2 of siz...,Hard,Binary Search;Divide and conquer,log(N),https://leetcode.com/problems/median-of-two-so...
4,"Given a string s, return the longest \npalindr...",Medium,Dynamic Programming,N^2,https://leetcode.com/problems/longest-palindro...
...,...,...,...,...,...
68,"Given an m x n integers matrix, return the len...",Hard,Dynamic Programming,N^2,https://leetcode.com/problems/longest-increasi...
69,Write a function that reverses a string. The i...,Easy,Implementation,N,https://leetcode.com/problems/reverse-string/
70,"Given an integer array nums and an integer k, ...",Medium,Hashmap,N,https://leetcode.com/problems/top-k-frequent-e...
71,"Given a string s, find the first non-repeating...",Easy,Hashmap,N,https://leetcode.com/problems/first-unique-cha...


In [62]:
def clean_text(text):
  text = text.strip()
  text = text.lower()

  lemmatizer = WordNetLemmatizer()
  words = text.split()
  words = [lemmatizer.lemmatize(word) for word in words]
  text = ' '.join(words)

  return text

nlp = spacy.load("en_core_web_sm")
def split_text_to_sentences(text):
    doc = nlp(text)

    sentences = [sentence.text for sentence in doc.sents]

    return sentences



In [ ]:
def clean_dataframe(data_frame):
    data_frame["Problem"] = data_frame["Problem"].apply(lambda problem: clean_text(problem))
    return data_frame

In [42]:
def split_dataframe_by_tag(data_frame):
    data_frame = data_frame.assign(Tag=data_frame['Tags'].str.split(';')).explode('Tag')
    return data_frame

In [219]:
data_frame = clean_dataframe(data_frame)


In [220]:
processed_dataframe, test_dataframe = train_test_split(data_frame, test_size = 0.2, random_state = 42)

In [223]:
test_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link
4,"given a string s, return the longest palindrom...",Medium,Dynamic Programming,N^2,https://leetcode.com/problems/longest-palindro...
63,given an array nums containing n distinct numb...,Easy,Bit manipulation,N,https://leetcode.com/problems/missing-number/
18,"given an array nums of distinct integers, retu...",Medium,Backtracking,N!,https://leetcode.com/problems/permutations/
0,given an array of integer nums and an integer ...,Easy,Brute force;Hashmap,N^2,https://leetcode.com/problems/two-sum/
28,given an m x n grid of character board and a s...,Medium,DFS,N^2,https://leetcode.com/problems/word-search/
72,"given four integer array nums1, nums2, nums3, ...",Medium,Hashmap;Binary Search,N^2,https://leetcode.com/problems/4sum-ii/descript...
10,given a string s containing just the character...,Easy,Stack,N,https://leetcode.com/problems/valid-parentheses/
34,you are given an integer array price where pri...,Medium,Dynamic Programming,N,https://leetcode.com/problems/best-time-to-buy...
12,you are given the head of two sorted linked li...,Easy,Linked List;Implementation,N,https://leetcode.com/problems/merge-two-sorted...
55,"given the head of a singly linked list, revers...",Easy,Implementation,N,https://leetcode.com/problems/reverse-linked-l...


In [221]:
processed_dataframe = split_dataframe_by_tag(processed_dataframe)

In [222]:
processed_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link,Tag
22,given an array of interval where intervals[i] ...,Medium,Implementation,NlogN,https://leetcode.com/problems/merge-intervals/,Implementation
57,"given an integer array nums and an integer k, ...",Medium,Priority Queue,NlogN,https://leetcode.com/problems/kth-largest-elem...,Priority Queue
50,"given a list of non-negative integer nums, arr...",Medium,Implementation,N^2logN,https://leetcode.com/problems/largest-number/s...,Implementation
33,you are given an array price where prices[i] i...,Easy,Dynamic Programming,N,https://leetcode.com/problems/best-time-to-buy...,Dynamic Programming
39,every adjacent pair of wordgiven an m x n matr...,Medium,BFS;DFS,N^2,https://leetcode.com/problems/surrounded-regions/,BFS
...,...,...,...,...,...,...
23,there is a robot on an m x n grid. the robot i...,Medium,Dynamic Programming,N^2,https://leetcode.com/problems/unique-paths/,Dynamic Programming
20,"given an integer array nums, find the subarray...",Medium,Dynamic Programming,N,https://leetcode.com/problems/maximum-subarray/,Dynamic Programming
60,"given an integer array nums, return an array a...",Medium,Dynamic Programming,N,https://leetcode.com/problems/product-of-array...,Dynamic Programming
14,"given two string needle and haystack, return t...",Easy,Sliding Window,N^2,https://leetcode.com/problems/find-the-index-o...,Sliding Window


In [87]:
def analyze_dataframe(data_frame):
  max_number_of_letters = 0
  max_number_of_words_per_sentence = 0
  max_number_of_sentences = 0
  vocabs = dict()
  problems_count_by_tag = defaultdict(int)
  problems_count_by_difficulty = defaultdict(int)
  problems_count_by_time_complexity = defaultdict(int)

  for row in processed_dataframe.iterrows():
      row_content = row[1]
      problem, tag, time_complexity, difficulty = row_content["Problem"], row_content["Tag"], row_content["Worst Complexity"], row_content["Difficulty"]

      max_number_of_letters = max(max_number_of_letters, len(problem))

      problem_sentences = split_text_to_sentences(problem)
      number_of_sentences = len(problem_sentences)
      max_number_of_sentences = max(max_number_of_sentences, number_of_sentences)

      for sentence in problem_sentences:
        words = sentence.split(" ")

        for word in words:
          vocabs[word] = 1

        number_of_words = len(words)
        max_number_of_words_per_sentence = max(max_number_of_words_per_sentence, number_of_words)

        problems_count_by_tag[tag] += 1
        problems_count_by_difficulty[difficulty] += 1
        problems_count_by_time_complexity[time_complexity] += 1

  return {
      "max_number_of_words_per_sentence": max_number_of_words_per_sentence,
      "max_number_of_sentences": max_number_of_sentences,
      "max_number_of_letters": max_number_of_letters,
      "vocabs": vocabs,
      "problems_count_by_tag": problems_count_by_tag,
      "problems_count_by_difficulty": problems_count_by_difficulty,
      "problems_count_by_time_complexity": problems_count_by_time_complexity
  }





In [88]:
analyze_result = analyze_dataframe(processed_dataframe)

In [90]:
analyze_result["max_number_of_letters"]

549

In [96]:
def get_tokenizer(texts):
    tokenizer = Tokenizer(num_words = VOCAB_SIZE)
    tokenizer.fit_on_texts(texts)
    return tokenizer

In [97]:
problems = processed_dataframe["Problem"]
tokenizer = get_tokenizer(problems)

In [ ]:
tokenizer

In [103]:
def permute_sentences(text):
    sentences = split_text_to_sentences(text)
    random.shuffle(sentences)  # Shuffle the sentences randomly
    permuted_text = '. '.join(sentences)  # Join the sentences back into a text string
    return permuted_text


In [113]:
# Permute the sentences in the problem to get a little more data
def permute_problem_sentences_in_dataframe(data_frame):
    new_rows = []
    for row in data_frame.iterrows():
        row_content = row[1]
        problem, tag, time_complexity, difficulty, link = row_content["Problem"], row_content["Tag"], row_content["Worst Complexity"], \
            row_content["Difficulty"], row_content["Link"]
        permuted_problem = permute_sentences(problem)
        new_row = {
            "Problem": permuted_problem,
            "Tag": tag,
            "Worst Complexity": time_complexity,
            "Difficulty": difficulty,
            "Link": link
        }
        new_rows.append(new_row)

    new_data_frame = data_frame.append(new_rows)
    return new_data_frame


In [224]:
processed_dataframe = permute_problem_sentences_in_dataframe(processed_dataframe)

<ipython-input-113-ce4e572044b7>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data_frame = data_frame.append(new_rows)


In [225]:
processed_dataframe

,Problem,Difficulty,Tags,Worst Complexity,Link,Tag
22,given an array of interval where intervals[i] ...,Medium,Implementation,NlogN,https://leetcode.com/problems/merge-intervals/,Implementation
57,"given an integer array nums and an integer k, ...",Medium,Priority Queue,NlogN,https://leetcode.com/problems/kth-largest-elem...,Priority Queue
50,"given a list of non-negative integer nums, arr...",Medium,Implementation,N^2logN,https://leetcode.com/problems/largest-number/s...,Implementation
33,you are given an array price where prices[i] i...,Easy,Dynamic Programming,N,https://leetcode.com/problems/best-time-to-buy...,Dynamic Programming
39,every adjacent pair of wordgiven an m x n matr...,Medium,BFS;DFS,N^2,https://leetcode.com/problems/surrounded-regions/,BFS
...,...,...,...,...,...,...
68,the test case are generated so that the answer...,Medium,NaN,N^2,https://leetcode.com/problems/unique-paths/,Dynamic Programming
69,"given an integer array nums, find the subarray...",Medium,NaN,N,https://leetcode.com/problems/maximum-subarray/,Dynamic Programming
70,you must write an algorithm that run in o(n) t...,Medium,NaN,N,https://leetcode.com/problems/product-of-array...,Dynamic Programming
71,"given two string needle and haystack, return t...",Easy,NaN,N^2,https://leetcode.com/problems/find-the-index-o...,Sliding Window


In [226]:
def encode_dataframe_column(data_frame, column_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(data_frame[column_name])
    type_mapping = {label: i for i, label in enumerate(label_encoder.classes_)}

    return {
        "type_mapping": type_mapping,
        "encoded_labels": encoded_labels
    }

In [227]:
tag_encoded_info = encode_dataframe_column(processed_dataframe, "Tag")
processed_dataframe["Tag"] = tag_encoded_info["encoded_labels"]

In [228]:
tag_encoded_info

{'type_mapping': {'BFS': 0,
  'Backtracking': 1,
  'Binary Search': 2,
  'Bit manipulation': 3,
  'DFS': 4,
  'Divide and conquer': 5,
  'Dynamic Programming': 6,
  'Greedy': 7,
  'Hashmap': 8,
  'Implementation': 9,
  'Linked List': 10,
  'Math': 11,
  'Merge sort': 12,
  'Priority Queue': 13,
  'Sliding Window': 14,
  'Sorting': 15,
  'Trie': 16},
 'encoded_labels': array([ 9, 13,  9,  6,  0,  4,  8,  2,  9,  1,  9, 14,  8,  0,  4,  3,  4,
         0,  6,  9,  9,  0,  4, 14,  6,  1, 13, 12,  6,  2,  5,  9,  6,  1,
         8, 16, 15,  9, 10,  9,  0,  4,  8, 11,  8,  2,  1,  7, 14,  2,  6,
         8,  6,  1,  4,  9,  3,  0,  4,  9,  6,  6, 10,  9,  3,  6, 14,  8,
         6,  6,  6, 14,  3,  9, 13,  9,  6,  0,  4,  8,  2,  9,  1,  9, 14,
         8,  0,  4,  3,  4,  0,  6,  9,  9,  0,  4, 14,  6,  1, 13, 12,  6,
         2,  5,  9,  6,  1,  8, 16, 15,  9, 10,  9,  0,  4,  8, 11,  8,  2,
         1,  7, 14,  2,  6,  8,  6,  1,  4,  9,  3,  0,  4,  9,  6,  6, 10,
         9,  3,  6, 14

In [229]:
complexity_encoded_info = encode_dataframe_column(processed_dataframe, "Worst Complexity")
processed_dataframe["Worst Complexity"] = complexity_encoded_info["encoded_labels"]


In [230]:
complexity_encoded_info

{'type_mapping': {'1': 0,
  '2^N': 1,
  '4^N': 2,
  'N': 3,
  'N*2^N': 4,
  'N^2': 5,
  'N^2logN': 6,
  'N^3': 7,
  'NlogN': 8,
  'log(N)': 9},
 'encoded_labels': array([8, 8, 6, 3, 5, 5, 3, 9, 3, 1, 3, 5, 5, 3, 3, 0, 3, 3, 3, 0, 5, 3,
        3, 3, 4, 4, 8, 8, 3, 9, 9, 3, 3, 2, 3, 5, 5, 3, 3, 3, 5, 5, 5, 5,
        3, 9, 1, 3, 3, 9, 3, 3, 3, 2, 3, 3, 3, 5, 5, 3, 7, 5, 3, 3, 0, 3,
        3, 3, 5, 3, 3, 5, 0, 8, 8, 6, 3, 5, 5, 3, 9, 3, 1, 3, 5, 5, 3, 3,
        0, 3, 3, 3, 0, 5, 3, 3, 3, 4, 4, 8, 8, 3, 9, 9, 3, 3, 2, 3, 5, 5,
        3, 3, 3, 5, 5, 5, 5, 3, 9, 1, 3, 3, 9, 3, 3, 3, 2, 3, 3, 3, 5, 5,
        3, 7, 5, 3, 3, 0, 3, 3, 3, 5, 3, 3, 5, 0])}

In [231]:
difficulty_encoded_info = encode_dataframe_column(processed_dataframe, "Difficulty")
processed_dataframe["Difficulty"] = difficulty_encoded_info["encoded_labels"]

In [233]:
difficulty_encoded_info

{'type_mapping': {'Easy': 0, 'Hard': 1, 'Medium': 2},
 'encoded_labels': array([2, 2, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2,
        2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1,
        0, 2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1, 0, 2, 1, 2, 2, 0, 2,
        2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2,
        0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0,
        0, 2, 2, 2, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 1, 1,
        0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0])}

In [236]:
problems = processed_dataframe["Problem"]
complexity = processed_dataframe["Worst Complexity"]
difficulty = processed_dataframe["Difficulty"]
tag = processed_dataframe["Tag"]

In [240]:
def convert_texts_to_vectors_of_sequences(texts, max_number_of_sentences, max_words_per_sentence, tokenizer):
    texts_tokenized = np.zeros((len(texts), max_number_of_sentences, max_words_per_sentence))

    for i, text in enumerate(texts):
        sentences = split_text_to_sentences(text)
        num_sentences = min(len(sentences), max_number_of_sentences)
        for j in range(num_sentences):
            sentence = sentences[j].strip()

            sentence_words = tokenizer.texts_to_sequences([sentence])[0]

            sentence_words = pad_sequences([sentence_words], maxlen=max_words_per_sentence, padding='post', truncating='post')[0]

            texts_tokenized[i, j, :] = sentence_words

        if num_sentences < max_number_of_sentences:
            texts_tokenized[i, num_sentences:, :] = np.zeros((max_number_of_sentences - num_sentences , max_words_per_sentence))

    return texts_tokenized

In [248]:
test_problem = problems[10]
tokenized_problem = convert_texts_to_vectors_of_sequences([test_problem], MAX_NUMBER_OF_SENTENCES, MAX_NUMBER_OF_WORDS_PER_SENTENCE, \
                                                          tokenizer)

In [250]:
tokenized_problem[0]

array([[103.,   8., 562., 375.,  10., 266.,  86.,   3.,  83.,  79.,   7.,
          8.,  51.,  47.,  18.,  15.,  73.,  73., 150.,  73., 119.,   1.,
        563., 564.,  15.,   7.,  38., 565.,  22.,  43.,   7., 121.,  34.,
         87.,  12.,  67.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [ 15.,   7.,  38., 566.,  22.,  43.,   7., 121.,  34., 177.,  12.,
        231.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,